In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("cerebras/btlm-3b-8k-base", torch_dtype=torch.float16)
model = AutoModelForCausalLM.from_pretrained("cerebras/btlm-3b-8k-base", torch_dtype=torch.float16, trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

...

A new version of the following files was downloaded from https://huggingface.co/cerebras/btlm-3b-8k-base:
- configuration_btlm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_btlm.py:   0%|          | 0.00/72.0k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/cerebras/btlm-3b-8k-base:
- modeling_btlm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin:   0%|          | 0.00/5.29G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [3]:
model = model.to('cuda:0')
model.generation_config.pad_token_id = model.generation_config.eos_token_id

In [15]:
def process_prompts(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as file:
        prompts = file.read().split('\n\n')

    with open(output_file, 'w', encoding='utf-8') as outfile:
        for i, prompt in enumerate(prompts):
            if prompt.strip():
                input_ids = tokenizer(prompt, return_tensors="pt").to(model.device)
                outputs = model.generate(
                    input_ids['input_ids'],
                    max_new_tokens=40,
                    early_stopping=True,
                    # num_return_sequences=1
                )
                response = tokenizer.decode(outputs[0])

                response = response.split('The correct answer is:')[1].strip()
                # response = response.split('Classify the sentiment of the following statement:')[1].strip()
                question = prompt.split('---')[1]
                outfile.write(f"Question {i+1}: {question}\n\nModel's response for question {i+1}: {response}\n------------------------------------------\n")
                print(f"Processed prompt {i+1}")
                print("\n")
                print(f"Question: {question}")
                print("\n")
                print(f"Response: {response}")
                print("---------------------------------")

    print("All responses have been successfully processed and saved.")

In [ ]:
input_file = '/content/drive/MyDrive/NLP Project/Data/Sentiment/sentiment_prompts.txt'
output_file = '/content/drive/MyDrive/NLP Project/Data/Sentiment/Responses/btlm_responses_for_sentiment.txt'

process_prompts(input_file, output_file)

In [ ]:
input_file = '/content/drive/MyDrive/NLP Project/Data/Sentiment/sentiment_prompts_random_label.txt'
output_file = '/content/drive/MyDrive/NLP Project/Data/Sentiment/Responses/btlm_responses_for_sentiment_random_label.txt'

process_prompts(input_file, output_file)

In [ ]:
input_file = '/content/drive/MyDrive/NLP Project/Data/Sentiment/sentiment_prompts_no_demos.txt'
output_file = '/content/drive/MyDrive/NLP Project/Data/Sentiment/Responses/btlm_responses_for_sentiment_no_demos.txt'

process_prompts(input_file, output_file)

In [ ]:
input_file = '/content/drive/MyDrive/NLP Project/Data/Sentiment/sentiment_prompts_cot.txt'
output_file = '/content/drive/MyDrive/NLP Project/Data/Sentiment/Responses/btlm_responses_for_sentiment_cot.txt'

process_prompts(input_file, output_file)

In [ ]:
input_file = '/content/drive/MyDrive/NLP Project/Data/random_arithmetic_prompts.txt'
output_file = '/content/drive/MyDrive/NLP Project/Data/btlm_responses_for_random_arithmetic.txt'

process_prompts(input_file, output_file)